In [1]:
from typing import *
import os

# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import tensorflow as tf
import tensorflow_probability as tfp

import numpy as np
import matplotlib.pyplot as plt

from scipy.stats import multivariate_normal
from scipy.stats import norm

from functools import partial
from time import time

tfd = tfp.distributions

base_dir = "/".join(os.getcwd().split("/")[:-1])

In [15]:
def vector_dot(a: tf.Tensor, b: tf.Tensor):
    return tf.reduce_sum(a*b, axis=-1)


def standard_normal_cdf_and_inverse_cdf(dtype: tf.DType):
    
    normal = tfp.distributions.Normal(
        loc=tf.zeros(shape=(), dtype=dtype),
        scale=tf.ones(shape=(), dtype=dtype),
    )
    Phi = lambda x: normal.cdf(x)
    iPhi = lambda x: normal.quantile(x)
    
    return Phi, iPhi


def get_update_indices(B: int, S: int, Q: int, q: int):
    
    dtype = tf.int32
    
    idxB = tf.tile(tf.range(B, dtype=dtype)[:, None, None], (1, S, 1))
    idxS = tf.tile(tf.range(S, dtype=dtype)[None, :, None], (B, 1, 1))
    idxQ = tf.tile(tf.convert_to_tensor(q)[None, None, None], (B, S, 1))
    
    idx = tf.concat([idxB, idxS, idxQ], axis=-1)
    
    return idx
    

def make_mvn_cdf(Q: int, S: int, dtype: tf.DType):
    
    # Draw the number of Sobol samples
    samples = tf.math.sobol_sample(dim=Q, num_results=S, dtype=dtype)

    @tf.function(jit_compile=True)
    def _mvn_cdf(
            mean: tf.Tensor,
            cov: tf.Tensor,
            lower: tf.Tensor,
            upper: tf.Tensor,
        ):
        """
        mean              : tf.Tensor, shape (B, Q)
        cov               : tf.Tensor, shape (B, Q, Q)
        lower             : tf.Tensor, shape (B, Q)
        upper             : tf.Tensor, shape (B, Q)
        """
        
        tf.debugging.assert_shapes(
            [
                (mean, ("B", "Q")),
                (cov, ("B", "Q", "Q")),
                (lower, ("B", "Q")),
                (upper, ("B", "Q")),
            ]
        )

        # Identify data type to use for all calculations
        dtype = mean.dtype
        B, Q = mean.shape

        # Compute Cholesky factors
        C = tf.linalg.cholesky(cov)  # (B, Q, Q)

        # Rename samples and limits for brevity
        w = samples  # (S, Q)
        a = lower  # (B, Q)
        b = upper  # (B, Q)

        # Initialise transformation variables
        d = tf.zeros(shape=(B, S, Q), dtype=dtype)
        e = tf.zeros(shape=(B, S, Q), dtype=dtype)
        f = tf.zeros(shape=(B, S, Q), dtype=dtype)
        y = tf.zeros(shape=(B, S, Q), dtype=dtype)

        # Initialise standard normal for computing CDFs
        Phi, iPhi = standard_normal_cdf_and_inverse_cdf(dtype=dtype)

        # Get update indices for convenience later
        idx = get_update_indices(B=B, S=S, Q=Q, q=0)

        # Compute transformation variables at the first step
        d_update = tf.tile(Phi(a[:, None, 0] / C[:, None, 0, 0]), (1, S))  # (B, S)
        d = tf.tensor_scatter_nd_add(tensor=d, indices=idx, updates=d_update)

        e_update = tf.tile(Phi(b[:, None, 0] / C[:, None, 0, 0]), (1, S))  # (B, S)
        e = tf.tensor_scatter_nd_add(tensor=e, indices=idx, updates=e_update)

        f_update = e_update - d_update  # (B, S)
        f = tf.tensor_scatter_nd_add(tensor=f, indices=idx, updates=f_update)

        for i in tf.range(1, Q):

            # # Compute indices to update y tensor
            # idx = get_update_indices(B=B, S=S, Q=Q, q=i-1)

            # Update y tensor
            y_update = iPhi(d[:, :, i-1] + w[None, :, i-1] * (e[:, :, i-1] - d[:, :, i-1]))
            y = tf.tensor_scatter_nd_add(tensor=y, indices=idx, updates=y_update)

            # Compute indices to update d, e and f tensors
            idx = get_update_indices(B=B, S=S, Q=Q, q=i)

            # Update d tensor
            d_update = Phi((a[:, None, i] - vector_dot(C[:, None, i, :i], y[:, :, :i])) / C[:, None, i, i])
            d = tf.tensor_scatter_nd_add(tensor=d, indices=idx, updates=d_update)

            # Update e tensor
            e_update = Phi((b[:, None, i] - vector_dot(C[:, None, i, :i], y[:, :, :i])) / C[:, None, i, i])
            e = tf.tensor_scatter_nd_add(tensor=e, indices=idx, updates=e_update)

            # Update f tensor
            f_update = (e[:, :, i] - d[:, :, i]) * f[:, :, i-1]
            f = tf.tensor_scatter_nd_add(tensor=f, indices=idx, updates=f_update)

        return tf.reduce_mean(f[:, :, -1], axis=-1)
    
    return _mvn_cdf


def mc_mvn_cdf(
        mean: tf.Tensor,
        cov: tf.Tensor,
        lower: tf.Tensor,
        upper: tf.Tensor,
        num_samples: int,
    ):
    
    samples = tfd.MultivariateNormalFullCovariance(
        loc=mean,
        covariance_matrix=cov,
    ).sample(sample_shape=[num_samples])
    
    lt = tf.reduce_all(tf.math.less(lower, samples), axis=1)
    gt = tf.reduce_all(tf.math.less(samples, upper), axis=1)
    
    return tf.cast(tf.math.logical_and(lt, gt), dtype=tf.float32)

In [18]:
# Set random seed
tf.random.set_seed(0)

# Set default data type
dtype = tf.float64

# Number of samples and dimension of Gaussian
B = 4000
S = 100
Q = 20

# Set up the mean and covariance
mean = tf.random.uniform((B, Q), dtype=dtype)
rand = tf.random.uniform((B, Q, Q), dtype=dtype) / Q**0.5
cov = 0. * tf.matmul(rand, rand, transpose_b=True) + tf.eye(Q, dtype=dtype)[None, :, :]

# Set up lower and upper integration limits
upper = tf.random.uniform((B, Q), dtype=dtype)
lower = float("-inf") * tf.ones_like(upper)

mvn_cdf = make_mvn_cdf(Q=Q, S=S, dtype=dtype)

cdfs = mvn_cdf(
    mean=mean,
    cov=cov,
    lower=lower,
    upper=upper,
)

%timeit _ = mvn_cdf( \
    mean=mean, \
    cov=cov, \
    lower=lower, \
    upper=upper, \
)

112 ms ± 19.8 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


# Batch multi-point improvement

In [19]:
def compute_bm(
        mean: tf.Tensor,
        threshold: tf.Tensor,
    ):
    """
    Arguments:
        mean: tf.Tensor, shape (B, Q)
        threshold: tf.Tensor, shape (B,)
        
    Returns:
        b: tf.Tensor, shape (B, Q, Q), b[B, K, Q]
        m: tf.Tensor, shape (B, Q, Q), m[B, K, Q]
    """
    
    # Unpack tensor shape and data type
    B, Q = mean.shape
    dtype = mean.dtype
    
    # Compute b tensor
    threshold = tf.tile(threshold[:, None], (1, Q))
    threshold = tf.linalg.diag(threshold) # (B, Q, Q)
    
    b = tf.zeros(shape=(B, Q, Q), dtype=dtype)
    b = b - threshold
    
    # Compute m tensor
    m = mean[:, None, :] - mean[:, :, None]  # (B, Q, Q)
    m = m - tf.linalg.diag(mean)  # (B, Q, Q)
    
    return b, m


def delta(
        idx: int,
        dim: int,
        B: int,
        transpose: bool,
        dtype: tf.DType
    ):
    
    o1 = tf.ones(shape=(B, idx, dim), dtype=dtype)
    z1 = tf.zeros(shape=(B, 1, dim), dtype=dtype)
    o2 = tf.ones(shape=(B, dim-idx-1, dim), dtype=dtype)
    
    delta = tf.concat([o1, z1, o2], axis=1)
    delta = tf.transpose(delta, perm=[0, 2, 1]) if transpose else delta
    
    return delta
    

def compute_Sigma(covariance: tf.Tensor):
    
    B, Q, _ = covariance.shape
    dtype = covariance.dtype
    
    Sigma = tf.zeros(shape=(B, Q, Q, Q))
    
    def compute_single_slice(q):
        
        diq = delta(q, Q, B, transpose=False, dtype=dtype)
        dqj = delta(q, Q, B, transpose=True, dtype=dtype)
        
        Sigma_ij = covariance[:, :, :]
        Sigma_iq = covariance[:, :, q:q+1]
        Sigma_qj = covariance[:, q:q+1, :]
        Sigma_qq = covariance[:, q:q+1, q:q+1]
        
        cov = Sigma_ij * diq * dqj - Sigma_iq * diq - Sigma_qj * dqj + Sigma_qq
        
        return cov
    
    Sigma = tf.map_fn(
        compute_single_slice,
        tf.range(Q),
        fn_output_signature=dtype,
    )
    
    Sigma = tf.transpose(Sigma, perm=[1, 0, 2, 3])
        
    return Sigma


def compute_p(
        m_reshaped: tf.Tensor,
        b_reshaped: tf.Tensor,
        Sigma_reshaped: tf.Tensor,
        mvn_cdf: Callable,
    ):
    
    # Unpack dtype and mean shape
    dtype = m_reshaped.dtype
    BQ, Q = m_reshaped.shape # (B*Q, Q)
    
    # Compute mean, covariance, lower and upper bounds for p mvn normal cdf
    p_cdf_mean = tf.zeros(shape=(BQ, Q), dtype=dtype)  # (B*Q, Q)
    p_cdf_cov = Sigma_reshaped # (B*Q, Q, Q)
    
    p_cdf_upper = b_reshaped - m_reshaped  # (B*Q, Q)
    p_cdf_lower = float("-inf") * tf.ones_like(p_cdf_upper)  # (B*Q, Q)
    
    # Compute MVN CDF
    p = mvn_cdf(
        mean=p_cdf_mean,
        cov=p_cdf_cov,
        lower=p_cdf_lower,
        upper=p_cdf_upper,
    )  # (B*Q,)
    
    p = tf.reshape(p, shape=(B, Q))  # (B, Q)
    
    return p


def compute_c(
        m_reshaped: tf.Tensor,
        b_reshaped: tf.Tensor,
        Sigma_reshaped: tf.Tensor,
    ):
    """
    Arguments:
        m_reshaped: tf.Tensor, shape (B*Q, Q)
        b_reshaped: tf.Tensor, shape (B*Q, Q)
        Sigma_reshaped: tf.Tensor, shape (B*Q, Q, Q)
    """
    
    # Check shapes of covariance tensor
    tf.debugging.assert_shapes(
        [
            (m_reshaped, ("BQ", "Q")),
            (b_reshaped, ("BQ", "Q")),
            (Sigma_reshaped, ("BQ", "Q", "Q")),
        ]
    )
    
    # Unpack tensor shape and data type
    BQ, Q = m_reshaped.shape
    dtype = m_reshaped.dtype
    
    # Compute difference between b and m tensors
    diff = b_reshaped - m_reshaped # (B*Q, Q)
    
    # Compute c, including the ith entry, which we want to remove
    cov_ratio = Sigma_reshaped / tf.linalg.diag_part(Sigma_reshaped)[:, :, None] # (B*Q, Q, Q)
    c = diff[:, None, :] - diff[:, :, None] * cov_ratio # (B*Q, Q, Q)
    
    # Remove the ith entry by masking c with a boolean mask with False across
    # the diagonal and True in the off-diagonal terms
    mask = tf.math.logical_not(tf.cast(tf.eye(Q, dtype=tf.int32), dtype=tf.bool))
    mask = tf.tile(mask[None, :, :], (c.shape[0], 1, 1))
    
    c = tf.ragged.boolean_mask(c, mask).to_tensor()
    
    return c


def compute_Sigmai_matrix(Sigma_reshaped):
    """
    Sigma_reshaped: tf.Tensor, shape (B*Q, Q, Q)
    """
    
    # Unpack tensor shape
    BQ, Q, _ = Sigma_reshaped.shape
    
    Sigma_uv = tf.tile(Sigma_reshaped[:, None, :, :], (1, Q, 1, 1))
    Sigma_iu = tf.tile(Sigma_reshaped[:, :, :, None], (1, 1, 1, Q))
    Sigma_iv = tf.tile(Sigma_reshaped[:, :, None, :], (1, 1, Q, 1))
    Sigma_ii = tf.linalg.diag_part(Sigma_reshaped)[:, :, None, None]
    
    Sigmai_whole = Sigma_uv - Sigma_iu * Sigma_iv / Sigma_ii
    
    def create_blocks(q):
        
        block1 = tf.concat(
            [
                Sigmai_whole[:, q, :q, :q],
                Sigmai_whole[:, q, q+1:, :q],
            ],
            axis=1,
        )
        
        block2 = tf.concat(
            [
                Sigmai_whole[:, q, :q, q+1:],
                Sigmai_whole[:, q, q+1:, q+1:],
            ],
            axis=1,
        )
        
        Sigmai_block = tf.concat([block1, block2], axis=2)
        
        return Sigmai_block
    
    Sigmai = tf.map_fn(
        create_blocks,
        tf.range(Q),
        fn_output_signature=Sigmai_whole.dtype,
    )
    Sigmai = tf.transpose(Sigmai, perm=[1, 0, 2, 3])
    
    return Sigmai


def compute_Phi(
        c: tf.Tensor,
        Sigmai: tf.Tensor,
        mvn_cdf: Callable,
    ):
    """
    Arguments:
        c: tf.Tensor, shape (B*Q, Q, Q-1)
        Sigmai: tf.Tensor, shape (B*Q, Q, Q, Q-1)
    """
    
    # Unpack tensor shape and data type
    BQ, Q, _, _ = Sigmai.shape
    dtype = Sigmai.dtype

    c_reshaped = tf.reshape(c, (BQ*Q, Q-1))
    Sigmai_reshaped = tf.reshape(Sigmai, (BQ*Q, Q-1, Q-1))
    
    # Compute mean, covariance, lower and upper bounds for Phi mvn normal cdf
    Phi_cdf_mean = tf.zeros(shape=(BQ*Q, Q-1), dtype=dtype)  # (B*Q*Q, Q)
    Phi_cdf_cov = Sigmai_reshaped  # (B*Q*Q, Q-1, Q-1)
    
    Phi_cdf_upper = c_reshaped  # (B*Q, Q-1)
    Phi_cdf_lower = float("-inf") * tf.ones_like(Phi_cdf_upper)  # (B*Q*Q, Q-1)
    
    # Compute multivariate cdfs
    mvn_cdfs = mvn_cdf(
        mean=Phi_cdf_mean,
        cov=Phi_cdf_cov,
        lower=Phi_cdf_lower,
        upper=Phi_cdf_upper,
    )
    mvn_cdfs = tf.reshape(mvn_cdfs, (B, Q, Q))  # (B, Q, Q)
    
    return mvn_cdfs


def compute_phi(
        Sigma: tf.Tensor,
        m: tf.Tensor,
        S_diag: tf.Tensor,
        b: tf.Tensor,
    ):
    
    S_diag = tf.linalg.diag_part(Sigma)
    normal = tfp.distributions.Normal(loc=m, scale=S_diag**0.5)
    phi = tf.math.exp(normal.log_prob(b))  # (B, Q, Q)
    
    return phi


@tf.function # (jit_compile=True)
def expected_improvement(
        mean: tf.Tensor,
        covariance: tf.Tensor,
        threshold: tf.Tensor,
        mvn_cdf: Callable,
    ):
    """
    Arguments:
        mean: tf.Tensor, shape (B, Q)
        covariance: tf.Tensor, shape (B, Q, Q)
        threshold: tf.Tensor, shape (B,)
    """
    
    timing = {}
    
    # Unpack dtype and mean shape
    dtype = mean.dtype
    B, Q = mean.shape
    
    # Compute b and m tensors
    t = time()
    b, m = compute_bm(mean=mean, threshold=threshold) # (B, Q, Q), (B, Q, Q)
    timing["compute_bm"] = time() - t
        
    # Compute Sigma
    t = time()
    Sigma = compute_Sigma(covariance=covariance) # (B, Q, Q, Q)
    timing["compute_Sigma"] = time() - t
    
    # Reshape all tensors, for batching
    b_reshaped = tf.reshape(b, (B*Q, Q))
    m_reshaped = tf.reshape(m, (B*Q, Q))
    Sigma_reshaped = tf.reshape(Sigma, (B*Q, Q, Q))
    
    # Compute p tensor
    t = time()
    p = compute_p(
        m_reshaped=m_reshaped,
        b_reshaped=b_reshaped,
        Sigma_reshaped=Sigma_reshaped,
        mvn_cdf=mvn_cdf,
    )
    timing["compute_p"] = time() - t
        
    # Compute c
    t = time()
    c = compute_c(
        m_reshaped=m_reshaped,
        b_reshaped=b_reshaped,
        Sigma_reshaped=Sigma_reshaped,
    ) # (B*Q, Q, Q-1)
    timing["compute_c"] = time() - t
        
    # Compute Sigma_i
    t = time()
    Sigmai = compute_Sigmai_matrix(
        Sigma_reshaped=Sigma_reshaped,
    ) # (B*Q, Q, Q-1, Q-1)
    timing["compute_Sigmai"] = time() - t
    
    # Compute Q-1 multivariate CDFs
    t = time()
    Phi_mvn_cdfs = compute_Phi(
        c=c,
        Sigmai=Sigmai,
        mvn_cdf=mvn_cdf,
    )
    timing["compute_Phi_mvn_cdfs"] = time() - t
        
    # Compute univariate pdfs
    t = time()
    phi = compute_phi(Sigma=Sigma, m=m, S_diag=S_diag, b=b)
    timing["compute_uvn_pdfs"] = time() - t
    
    Sigma_diag = tf.linalg.diag_part(
        tf.transpose(Sigma, perm=[0, 2, 1, 3])
    )
    Sigma_diag = tf.transpose(Sigma_diag, perm=[0, 2, 1])
    
    T = tf.tile(threshold[:, None], (1, Q))
    
    mean_T_term = (mean - T) * p
    
    sum_term = tf.reduce_sum(
        Sigma_diag * uvn_pdfs * Phi_mvn_cdfs,
        axis=2,
    )
    
    qEI = tf.reduce_sum(mean_T_term + sum_term, axis=1)
    
    return qEI, timing

In [22]:
tf.random.set_seed(0)

B = 10
S = 100
Q = 30

mvn_cdf = make_mvn_cdf(Q=Q, S=S, dtype=dtype)

# mean = tf.random.uniform(shape=(B, Q), dtype=dtype)
mean = tf.zeros(shape=(B, Q), dtype=dtype)
rand = tf.random.uniform((B, Q, Q), dtype=dtype) / Q**0.5
covariance = 1e-1 * tf.matmul(rand, rand, transpose_b=True) + tf.eye(Q, dtype=dtype)[None, ...]

threshold = tf.random.uniform((B,), dtype=dtype)

_ = expected_improvement(
    mean=mean,
    covariance=covariance,
    threshold=threshold,
    mvn_cdf=mvn_cdf,
)

qEI, timing = expected_improvement(
    mean=mean,
    covariance=covariance,
    threshold=threshold,
    mvn_cdf=mvn_cdf,
)

In [2]:
import tensorflow as tf

tf.math.maximum(tf.convert_to_tensor([0., 1.]), 0.5)

2022-10-18 16:19:25.979548: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-18 16:19:26.733501: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9652 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:17:00.0, compute capability: 7.5
2022-10-18 16:19:26.734098: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 9652 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:b3:00.0, compute capability: 7.5


<tf.Tensor: shape=(2,), dtype=float32, numpy=array([0.5, 1. ], dtype=float32)>

In [ ]:
from trieste